In [1]:
# Generic imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import dask.dataframe as dd
from dask.distributed import Client

client = Client(n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:41105 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


In [3]:
# Raw dataframe loaded from csv file
df_on_load = pd.read_csv('gun-violence-data_01-2013_03-2018.csv')

In [4]:
# Adding Las Vegas incident
vegas_shotout = [000000, '2017-10-01', 'Nevada', 'Las Vegas', 'Mandalay Bay 3950 Blvd S', 59, 489, 'https://en.wikipedia.org/wiki/2017_Las_Vegas_shooting', 'https://en.wikipedia.org/wiki/2017_Las_Vegas_shooting', '-', '-', '-', '-', 'mass', 36.095, 'Hotel', 
              -115.171667, 47, 'Route 91 Harvest Festiva; concert, mass open fire from 32nd floor. 47 guns seized; TOTAL:59 kill, 489 inj, number shot TBD,girlfriend Marilou Danley POI', '0::47', '0::Adult 18+', '0::Male', '0::Stephen Paddock', '0::Killed', '0::Dead', '0::Subject-Suspect', 'https://en.wikipedia.org/wiki/2017_Las_Vegas_shooting', '1', '4']
df_on_load.loc[len(df_on_load)] = vegas_shotout

In [5]:
pd.set_option('display.max_columns', None)  # Unlimited columns
df_on_load.head(2)

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62,35


In [6]:
# Dropping columns which are of less significance for analysis
df_significant = \
    df_on_load.drop(columns=["source_url",
                             "incident_url_fields_missing",
                             "congressional_district",
                             "location_description",
                             "sources",
                             "notes",
                             "state_house_district",
                             "state_senate_district"])

In [7]:
df_significant.isnull().sum()

incident_id                      0
date                             0
state                            0
city_or_county                   0
address                      16497
n_killed                         0
n_injured                        0
incident_url                     0
gun_stolen                   99498
gun_type                     99451
incident_characteristics       326
latitude                      7923
longitude                     7923
n_guns_involved              99451
participant_age              92298
participant_age_group        42119
participant_gender           36362
participant_name            122253
participant_relationship    223903
participant_status           27626
participant_type             24863
dtype: int64

In [8]:
def consolidate(row):
    """
    Consolidates the information across different participant features
    together.
    """
    participant_info = {}
  
    def consolidate_info(value, value_key):
        nonlocal participant_info

        if type(value) is not str:
          return
    
        participants = value.split("||")

        for participant in participants:
            info_key = participant.split("::")[0]
            info_value = participant.split("::")[1]

            if info_key in participant_info:
                existing_info = participant_info[info_key]
                existing_info[value_key] = info_value
                participant_info[info_key] = existing_info
            else:
                participant_info[info_key] = {value_key : info_value}
  
    age = row['participant_age']
    age_group = row["participant_age_group"]
    gender = row["participant_gender"]
    name = row["participant_name"]
    relationship = row["participant_relationship"]
    status = row["participant_status"]
    ptype = row["participant_type"]

    consolidate_info(age, "age")
    consolidate_info(age_group, "age_group")
    consolidate_info(gender, "gender")
    consolidate_info(name, "name")
    consolidate_info(relationship, "relationship")
    consolidate_info(status, "status")
    consolidate_info(ptype, "type")

    return participant_info

def cleanup(data):
    """
    Cleaning up data with inconsistent separator encoding.
    """
    if (type(data) is str):
        if data.find("||") == -1 and data.find("|") != -1:
            data = data.replace("|", "||")
    
        if data.find("::") == -1 and data.find(":") != -1:
            data = data.replace(":", "::")
  
    return data


def updatemissing(row, column, lat_long_df):
    """Update missing latitude or longitude information with information
    fetched from Geocoder."""
    import math
    
    if not math.isnan(row[column]):
        return row[column]
    
    incident_id = row['incident_id']
    matched_df = lat_long_df.loc[lat_long_df['incident_id'] == incident_id]
    matched_series = matched_df.loc[:,[column]][column]
    if matched_series.empty:
        return row[column]
    
    return matched_series.values[0]

In [9]:
def feature_engineering(df):
    # clean up participant information
    df['participant_age'] = list(map(cleanup, df['participant_age']))
    df['participant_age_group'] = list(map(cleanup, df['participant_age_group']))
    df['participant_gender'] = list(map(cleanup, df['participant_gender']))
    df['participant_name'] = list(map(cleanup, df['participant_name']))
    df['participant_relationship'] = \
    list(map(cleanup, df['participant_relationship']))
    df['participant_status'] = list(map(cleanup, df['participant_status']))
    df['participant_type'] = list(map(cleanup, df['participant_type']))

    # Consolidate participant information
    df['participant_info'] = df.apply(consolidate, axis=1)

    lat_long_df = pd.read_csv("lat_long.csv")

    # Update missing latitude and longitude information
    df['latitude'] = df.apply(updatemissing, axis=1,
                              args=("latitude",lat_long_df,))
    df['longitude'] = df.apply(updatemissing, axis=1,
                              args=("longitude",lat_long_df,))
    
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day_of_month'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.weekday

    df['incident_type'] = df.incident_characteristics.str.replace(r'((?i)^.*mass.*$)', 'Mass Shooting')
    df['incident_type'] = df.incident_characteristics.str.replace(r'((?i)^.*murder.*$)', 'Homicide')
    df['incident_type'] = df.incident_characteristics.str.replace(r'((?i)^.*police.*$)', 'Police Involved')
    df['incident_type'] = df.incident_characteristics.str.replace(r'((?i)^.*officer.*$)', 'Police Involved')
    df['incident_type'] = df.incident_characteristics.str.replace(r'((?i)^.*accident.*$)', 'Accidental')
    df['incident_type'] = df.incident_characteristics.str.replace(r'((?i)^.*child.*$)', 'Child Involved')
    df['incident_type'] = df.incident_characteristics.str.replace(r'((?i)^.*suicide.*$)', 'Suicide')

In [10]:
feature_engineering(df_significant)

In [11]:
df_significant.isnull().sum()

incident_id                      0
date                             0
state                            0
city_or_county                   0
address                      16497
n_killed                         0
n_injured                        0
incident_url                     0
gun_stolen                   99498
gun_type                     99451
incident_characteristics       326
latitude                         0
longitude                        0
n_guns_involved              99451
participant_age              92298
participant_age_group        42119
participant_gender           36362
participant_name            122253
participant_relationship    223903
participant_status           27626
participant_type             24863
participant_info                 0
year                             0
month                            0
day_of_month                     0
day_of_week                      0
incident_type                  326
dtype: int64

In [12]:
!pip install geocoder

fastparquet 0.2.1 requires pytest-runner, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
import geocoder
from time import sleep
import csv

lat_long_df = pd.DataFrame(columns=['incident_id','latitude','longitude'])

def geocoding(df):
    
    # Update your API key here
    LOCATIONIQ_API_KEY=""

    for i in range(df.shape[0]):
    
        # The index are based on the specific df passed as argument.
        incident_id = df.iloc[i, 0]     # incident_id = 0
        state = df.iloc[i, 2]           # state = 2
        city_or_county = df.iloc[i, 3]  # city_or_county = 3
        address = df.iloc[i, 4]         # address = 4
        latitude = df.iloc[i, 11]       # latitude = 11
        longitude = df.iloc[i, 12]      # longitude = 12
    
        # It is seen both latitude and longitude are NaN together.
        # So checking for latitude is enough
        if not np.isnan(latitude):
            lat_long_df.loc[i] = [incident_id, latitude, longitude]
            continue
    
        lookup_string = ""
    
        if type(address) is str:
            lookup_string = address+','+city_or_county+','+state
      
            geocode_liq = geocoder.locationiq(lookup_string, key=LOCATIONIQ_API_KEY)
            if geocode_liq.json and geocode_liq.json['status'] == 'OK':
                latitude = geocode_liq.json['lat'] 
                longitude = geocode_liq.json['lng']
                print(f'{incident_id},{latitude},{longitude}')
            else:
                lookup_string = city_or_county+','+state
        
                geocode_liq = geocoder.locationiq(lookup_string, key=LOCATIONIQ_API_KEY)
                if geocode_liq.json and geocode_liq.json['status'] == 'OK':
                    latitude = geocode_liq.json['lat'] 
                    longitude = geocode_liq.json['lng']
                    print(f'{incident_id},{latitude},{longitude}')
        else:
            lookup_string = city_or_county+','+state
      
            geocode_liq = geocoder.locationiq(lookup_string, key=LOCATIONIQ_API_KEY)
            if geocode_liq.json and geocode_liq.json['status'] == 'OK':
                latitude = geocode_liq.json['lat'] 
                longitude = geocode_liq.json['lng']
                print(f'{incident_id},{latitude},{longitude}')
        
        lat_long_df.loc[i] = [incident_id, latitude, longitude]
        sleep(1) # End of for
  
    export_csv = lat_long_df.to_csv ('lat_long_df.csv', index = False, 
                                   header=True)
  

### Handling missing latitude and longitude information.

If any of the latitude or longitude inforamtion are missing,
uncomment and use this below code.

Note: LOCATIONIQ_API_KEY needs to be updated.
LOCATIONIQ API has a limit of 10000 requests per day.

**Output:**
**lat_long_df.csv** - contains the merged list of latitude and longitude.
**screen output**   - contains the list of missed latitude and longitude.

**Format:**
incident_id,latitude,longitude

In [14]:
#geocoding(df_significant)

In [15]:
df_significant.to_csv ('gun-violence-data_01-2013_03-2018_cleaned.csv', index = False, 
                                   header=True)

In [16]:
test_df = pd.read_csv("gun-violence-data_01-2013_03-2018_cleaned.csv")

In [17]:
test_df.tail()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,gun_stolen,gun_type,incident_characteristics,latitude,longitude,n_guns_involved,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,participant_info,year,month,day_of_month,day_of_week,incident_type
239673,1083139,2018-03-31,Louisiana,Natchitoches,247 Keyser Ave,1,0,http://www.gunviolencearchive.org/incident/108...,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)||Ins...",31.7537,-93.083600,1.0,1::21,0::Adult 18+||1::Adult 18+,0::Male||1::Male,0::Jamal Haskett||1::Jaquarious Tyjuan Ardison,NaN,"0::Killed||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,"{'1': {'age': '21', 'age_group': 'Adult 18+', ...",2018,3,31,5,Suicide
239674,1083151,2018-03-31,Louisiana,Gretna,1300 block of Cook Street,0,1,http://www.gunviolencearchive.org/incident/108...,0::Unknown,0::Unknown,Shot - Wounded/Injured,29.9239,-90.044200,1.0,0::21,0::Adult 18+,0::Male,NaN,NaN,0::Injured,0::Victim,"{'0': {'age': '21', 'age_group': 'Adult 18+', ...",2018,3,31,5,Shot - Wounded/Injured
239675,1082514,2018-03-31,Texas,Houston,12630 Ashford Point Dr,1,0,http://www.gunviolencearchive.org/incident/108...,0::Unknown,0::Unknown,"Shot - Dead (murder, accidental, suicide)",29.7201,-95.611000,1.0,0::42,0::Adult 18+,0::Male,0::Leroy Ellis,NaN,0::Killed,0::Victim,"{'0': {'age': '42', 'age_group': 'Adult 18+', ...",2018,3,31,5,Suicide
239676,1081940,2018-03-31,Maine,Norridgewock,434 Skowhegan Rd,2,0,http://www.gunviolencearchive.org/incident/108...,0::Unknown||1::Unknown,0::Handgun||1::Shotgun,"Shot - Dead (murder, accidental, suicide)||Sui...",44.7293,-69.769100,2.0,0::58||1::62,0::Adult 18+||1::Adult 18+,0::Female||1::Male,0::Marie Lancaster Hale||1::William Hale,1::Significant others - current or former,0::Killed||1::Killed,0::Victim||1::Subject-Suspect,"{'0': {'age': '58', 'age_group': 'Adult 18+', ...",2018,3,31,5,Suicide
239677,0,2017-10-01,Nevada,Las Vegas,Mandalay Bay 3950 Blvd S,59,489,https://en.wikipedia.org/wiki/2017_Las_Vegas_s...,-,-,mass,36.0950,-115.171667,47.0,0::47,0::Adult 18+,0::Male,0::Stephen Paddock,0::Killed,0::Dead,0::Subject-Suspect,"{'0': {'age': '47', 'age_group': 'Adult 18+', ...",2017,10,1,6,mass


In [18]:
test_df.isnull().sum()

incident_id                      0
date                             0
state                            0
city_or_county                   0
address                      16497
n_killed                         0
n_injured                        0
incident_url                     0
gun_stolen                   99498
gun_type                     99451
incident_characteristics       326
latitude                         0
longitude                        0
n_guns_involved              99451
participant_age              92298
participant_age_group        42119
participant_gender           36362
participant_name            122253
participant_relationship    223903
participant_status           27626
participant_type             24863
participant_info                 0
year                             0
month                            0
day_of_month                     0
day_of_week                      0
incident_type                  326
dtype: int64

distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34658 remote=tcp://127.0.0.1:41105>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34662 remote=tcp://127.0.0.1:41105>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34666 remote=tcp://127.0.0.1:41105>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34668 remote=tcp://127.0.0.1:41105>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34670 remote=tcp://127.0.0.1:41105>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34672 remote=tcp://127.0.0.1:41105>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34674 remote=tcp://127.0.0.1:41105>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP local=tcp://127.0.0.1:34676 remote=tcp://127.0.0.1:41105>
distributed.comm